In [ ]:
# Read processed_things.json

import json
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import datetime
import folium
import os
import math

def get_coordinates_by_thing_name():
    with open(f'things.json', 'r', encoding='utf-8') as JSON:
        things = json.load(JSON)
    thing_locations = {}
    for thing in things:
        location = None
        try:
            location = thing["Locations"][0]["location"]["geometry"]["coordinates"][0][0]
        except:
            location_found = False
            # Try to extract location from observed area from datastreams
            for datastream in thing["Datastreams"]:
                if "observedArea" not in datastream:
                    continue
                if not isinstance(datastream["observedArea"]["coordinates"][0], list):
                    location = datastream["observedArea"]["coordinates"]
                else:
                    location = datastream["observedArea"]["coordinates"][0]
                location_found = True
                break
            if not location_found:
                with open(f"last_thing_with_no_location_data.json", 'w') as fp:
                    json.dump(thing, fp)
                print(f"Location not found for thing {thing['name']}. Look at last_thing_with_no_location_data.json for more information.")
        if location is None:
            continue
        if isinstance(location[0], list):
            location = location[0]
        thing_locations[thing["name"]] = location
    return thing_locations

with open('processed_things.json') as f:
    processed_things = json.load(f)
    
thing_locations = get_coordinates_by_thing_name()

max_total_primary_signal_missing_count = 0
max_total_cycle_second_missing_count = 0
max_total_skipped_cycles_count = 0
max_total_removed_cycles_count = 0
max_total_cycles_count = 0
max_median_metric = 0
    
for thing_name, thing in processed_things.items():
    if thing['PrimarySignalMissingCount'] > max_total_primary_signal_missing_count:
        max_total_primary_signal_missing_count = thing['PrimarySignalMissingCount']
    if thing['CycleSecondMissingCount'] > max_total_cycle_second_missing_count:
        max_total_cycle_second_missing_count = thing['CycleSecondMissingCount']
    if thing['TotalSkippedCycles'] > max_total_skipped_cycles_count:
        max_total_skipped_cycles_count = thing['TotalSkippedCycles']
    if thing['TotalRemovedCycleCount'] > max_total_removed_cycles_count:
        max_total_removed_cycles_count = thing['TotalRemovedCycleCount']
    if thing['TotalCyclesCount'] > max_total_cycles_count:
        max_total_cycles_count = thing['TotalCyclesCount']
            
    values = []
    for day in thing['Metrics']:
        for hour in day:
            if hour != -1.0:
                values.append(hour)
            
    median = np.median(np.array(values))
    if median > max_median_metric:
        max_median_metric = median
        
primary_signal_missing_f = folium.Figure(width=600, height=500)
cycle_second_missing_f = folium.Figure(width=600, height=500)
skipped_cycles_f = folium.Figure(width=600, height=500)
removed_cycles_f = folium.Figure(width=600, height=500)
total_cycles_f = folium.Figure(width=600, height=500)
median_metric_f = folium.Figure(width=600, height=500)
primary_signal_missing_map = folium.Map(location=[53.57532, 10.01534], zoom_start=10).add_to(primary_signal_missing_f)
cycle_second_missing_map = folium.Map(location=[53.57532, 10.01534], zoom_start=10).add_to(cycle_second_missing_f)
skipped_cycles_map = folium.Map(location=[53.57532, 10.01534], zoom_start=10).add_to(skipped_cycles_f)
removed_cycles_map = folium.Map(location=[53.57532, 10.01534], zoom_start=10).add_to(removed_cycles_f)
total_cycles_map = folium.Map(location=[53.57532, 10.01534], zoom_start=10).add_to(total_cycles_f)
median_metric_map = folium.Map(location=[53.57532, 10.01534], zoom_start=10).add_to(median_metric_f)

folium.TileLayer('cartodbpositron').add_to(primary_signal_missing_map)
folium.TileLayer('cartodbpositron').add_to(cycle_second_missing_map)
folium.TileLayer('cartodbpositron').add_to(skipped_cycles_map)
folium.TileLayer('cartodbpositron').add_to(removed_cycles_map)
folium.TileLayer('cartodbpositron').add_to(total_cycles_map)
folium.TileLayer('cartodbpositron').add_to(median_metric_map)

for thing_name, thing in processed_things.items():
    primary_signals_missing_percentage = thing['PrimarySignalMissingCount'] / max_total_primary_signal_missing_count
    cycle_seconds_missing_percentage = thing['CycleSecondMissingCount'] / max_total_cycle_second_missing_count
    skipped_cycles_percentage = thing['TotalSkippedCycles'] / max_total_skipped_cycles_count if max_total_skipped_cycles_count != 0 else 0
    removed_cycles_percentage = thing['TotalRemovedCycleCount'] / thing['TotalCyclesCount'] if thing['TotalCyclesCount'] != 0 else 0
    total_cycles_percentage = thing['TotalCyclesCount'] / max_total_cycles_count
    
    primary_signals_missing_color = '#%02x%02x%02x' % (int(255 * primary_signals_missing_percentage), int(255 * (1 - primary_signals_missing_percentage)), 0)
    cycle_seconds_missing_color = '#%02x%02x%02x' % (int(255 * cycle_seconds_missing_percentage), int(255 * (1 - cycle_seconds_missing_percentage)), 0)
    skipped_cycles_color = '#%02x%02x%02x' % (int(255 * skipped_cycles_percentage), int(255 * (1 - skipped_cycles_percentage)), 0)
    removed_cycles_color = '#%02x%02x%02x' % (int(255 * removed_cycles_percentage), int(255 * (1 - removed_cycles_percentage)), 0)
    total_cycles_color = '#%02x%02x%02x' % (int(255 * total_cycles_percentage), int(255 * (1 - total_cycles_percentage)), 0)
    
    values = []
    for day in thing['Metrics']:
        for hour in day:
            if hour != -1.0:    
                values.append(hour)
            
    median = np.median(np.array(values))
    
    median_percentage = median / max_median_metric
    if math.isnan(median_percentage):
        median_color = '#%02x%02x%02x' % (0, 0, 0)
    else:
        median_color = '#%02x%02x%02x' % (int(255 * median_percentage), int(255 * (1 - median_percentage)), 0)
    
    try:
        coordinates = [thing_locations[thing_name][1],thing_locations[thing_name][0]]
    except:
        default_coordinates = [53.683899,9.867225]
        # randomize default coordinates
        default_coordinates[0] += np.random.uniform(-0.0001, 0.0001)
        default_coordinates[1] += np.random.uniform(-0.0001, 0.0001)
        coordinates = [default_coordinates[1],default_coordinates[0]]
    
    total_cycles_layer_element = folium.CircleMarker(coordinates, radius=3, color=total_cycles_color)
    primary_signals_missing_layer_element = folium.CircleMarker(coordinates, radius=3, color=primary_signals_missing_color)
    cycle_seconds_missing_layer_element = folium.CircleMarker(coordinates, radius=3, color=cycle_seconds_missing_color)
    skipped_cycles_layer_element = folium.CircleMarker(coordinates, radius=3, color=skipped_cycles_color)
    removed_cycles_layer_element = folium.CircleMarker(coordinates, radius=3, color=removed_cycles_color)
    median_metric_layer_element = folium.CircleMarker(coordinates, radius=3, color=median_color)
        
    popup_text = f"""
    <b>Thing: {thing_name}</b><br>
    Primary signal missing count: {thing['PrimarySignalMissingCount']}({primary_signals_missing_percentage})<br>
    Cycle second missing count: {thing['CycleSecondMissingCount']}({cycle_seconds_missing_percentage})<br>
    Skipped cycles: {thing['TotalSkippedCycles']}({skipped_cycles_percentage})<br>
    Removed cycles: {thing['TotalRemovedCycleCount']}({removed_cycles_percentage})<br>
    Total cycles: {thing['TotalCyclesCount']}({total_cycles_percentage})<br>
    Median metric: {median}({median_percentage})<br>
    """
    
    folium.Popup(popup_text, max_width=300,min_width=300).add_to(total_cycles_layer_element)
    folium.Popup(popup_text, max_width=300,min_width=300).add_to(primary_signals_missing_layer_element)
    folium.Popup(popup_text, max_width=300,min_width=300).add_to(cycle_seconds_missing_layer_element)
    folium.Popup(popup_text, max_width=300,min_width=300).add_to(skipped_cycles_layer_element)
    folium.Popup(popup_text, max_width=300,min_width=300).add_to(removed_cycles_layer_element)
    folium.Popup(popup_text, max_width=300,min_width=300).add_to(median_metric_layer_element)
    
    total_cycles_layer_element.add_to(total_cycles_map)
    primary_signals_missing_layer_element.add_to(primary_signal_missing_map)
    cycle_seconds_missing_layer_element.add_to(cycle_second_missing_map)
    skipped_cycles_layer_element.add_to(skipped_cycles_map)
    removed_cycles_layer_element.add_to(removed_cycles_map)
    median_metric_layer_element.add_to(median_metric_map)

# folium.LayerControl(collapsed=False).add_to(m)

title_html = '''
            <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''

title = "Primary signal missing count per thing"
title = title_html.format(title)
primary_signal_missing_map.get_root().html.add_child(folium.Element(title))
display(primary_signal_missing_map)

title = "Cycle second missing count per thing"
title = title_html.format(title)
cycle_second_missing_map.get_root().html.add_child(folium.Element(title))
display(cycle_second_missing_map)

title = "Skipped cycles per thing"
title = title_html.format(title)
skipped_cycles_map.get_root().html.add_child(folium.Element(title))
display(skipped_cycles_map)

title = "Removed cycles per thing"
title = title_html.format(title)
removed_cycles_map.get_root().html.add_child(folium.Element(title))
display(removed_cycles_map)

title = "Total cycles per thing"
title = title_html.format(title)
total_cycles_map.get_root().html.add_child(folium.Element(title))
display(total_cycles_map)

title = "Median metric per thing"
title = title_html.format(title)
median_metric_map.get_root().html.add_child(folium.Element(title))
display(median_metric_map)

    